In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv('../small1/labeled.csv')
df2 = pd.read_csv('../small2/labeled.csv')
df3 = pd.read_csv('../small3/labeled.csv')

In [3]:
df = pd.concat([df1, df2, df3])

In [4]:
df.head()

,title,is_clickbait,dataset
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1,fake-news
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0,fake-news
2,Why the Truth Might Get You Fired,1,fake-news
3,15 Civilians Killed In Single US Airstrike Hav...,1,fake-news
4,Iranian woman jailed for fictional unpublished...,1,fake-news


In [5]:
# drop empty titles
df = df[df['title'].notna()]

In [6]:
# drop duplicate titles
df = df.drop_duplicates(subset='title')

In [7]:
df = df.reset_index(drop=True)
df

,title,is_clickbait,dataset
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1,fake-news
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0,fake-news
2,Why the Truth Might Get You Fired,1,fake-news
3,15 Civilians Killed In Single US Airstrike Hav...,1,fake-news
4,Iranian woman jailed for fictional unpublished...,1,fake-news
...,...,...,...
74119,"To Make Female Hearts Flutter in Iraq, Throw a...",0,clickbait-dataset
74120,"British Liberal Democrat Patsy Calton, 56, die...",0,clickbait-dataset
74121,Drone smartphone app to help heart attack vict...,0,clickbait-dataset
74122,"Netanyahu Urges Pope Benedict, in Israel, to D...",0,clickbait-dataset


In [9]:
df.to_csv('../../modelling/data/merged_titles_labels.csv', index=False)